In [2]:
import folium
import geopandas as gpd

# Load the intersections from the GeoJSON file
file_path = "tram_intersections.geojson"  # Make sure the file exists
intersections = gpd.read_file(file_path)

# Get the central point of the map (mean of coordinates)
center_lat = intersections.geometry.y.mean()
center_lon = intersections.geometry.x.mean()

# Create a Folium map centered on Bratislava
m = folium.Map(
    location=[center_lat, center_lon], zoom_start=14, tiles="CartoDB Positron"
)

# Add tram track intersections as markers
for _, row in intersections.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
        popup=f"Intersection ID: {row.name}",
    ).add_to(m)

# Save map to HTML file
map_file = "tram_intersections_map.html"
m.save(map_file)

print(f"Map saved as {map_file}. Open it in a browser to view.")

# Optional: Show in Jupyter Notebook (if running in Jupyter)
m

Map saved as tram_intersections_map.html. Open it in a browser to view.


In [8]:
import osmnx as ox
import geopandas as gpd
import folium
from shapely.geometry import Point, LineString

# Define the city
place_name = "Bratislava, Slovakia"

# 1. Get all roads (not just driveable ones)
road_graph = ox.graph_from_place(place_name, network_type="all")
road_nodes, road_edges = ox.graph_to_gdfs(road_graph)

# 2. Get tram tracks
tram_graph = ox.graph_from_place(
    place_name, network_type="all", custom_filter='["railway"="tram"]'
)
_, tram_edges = ox.graph_to_gdfs(tram_graph)

# Convert both to the same coordinate reference system (for spatial operations)
road_edges = road_edges.to_crs(epsg=3857)
tram_edges = tram_edges.to_crs(epsg=3857)

# 3. Use a spatial join to find potential intersections
intersections = gpd.sjoin(road_edges, tram_edges, predicate="intersects")

# 4. Extract actual intersection points from overlapping roads/tram tracks
intersection_points = []
for geom in intersections.geometry:
    if isinstance(geom, LineString):  # Ensure valid points
        intersection_points.extend([Point(coord) for coord in geom.coords])

# Convert to GeoDataFrame
intersections_gdf = gpd.GeoDataFrame(geometry=intersection_points, crs="EPSG:3857")

# 5. Filter: Keep only points **inside a tram track buffer**
buffer_size = 10  # 10 meters
tram_buffer = tram_edges.buffer(buffer_size)  # Create buffer around tram tracks

filtered_intersections = intersections_gdf[
    intersections_gdf.geometry.within(tram_buffer.unary_union)
]

# Convert back to latitude/longitude
filtered_intersections = filtered_intersections.to_crs(epsg=4326)

# Remove duplicates
filtered_intersections = filtered_intersections.drop_duplicates()

# Save to GeoJSON
output_file = "road_tram_intersections_filtered.geojson"
filtered_intersections.to_file(output_file, driver="GeoJSON")

# 6. Display on Folium Map
center_lat, center_lon = (
    filtered_intersections.geometry.y.mean(),
    filtered_intersections.geometry.x.mean(),
)
m = folium.Map(
    location=[center_lat, center_lon], zoom_start=14, tiles="CartoDB Positron"
)

# Add filtered intersection points
for _, row in filtered_intersections.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
    ).add_to(m)

# Save and show map
map_file = "road_tram_intersections_filtered_map.html"
m.save(map_file)

print(f"Filtered data saved to {output_file}")
print(f"Map saved as {map_file}. Open it in a browser to view.")

# Optional: Show in Jupyter Notebook
m

/var/folders/bf/yx_hc41d1jl40qt0032h3_300000gn/T/ipykernel_54709/3225684902.py:40: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  intersections_gdf.geometry.within(tram_buffer.unary_union)


Filtered data saved to road_tram_intersections_filtered.geojson
Map saved as road_tram_intersections_filtered_map.html. Open it in a browser to view.


In [ ]:
import osmnx as ox
import geopandas as gpd
import folium
from shapely.geometry import Point, LineString

# Define city
place_name = "Bratislava, Slovakia"

# 1. Load all roads (including minor roads)
road_graph = ox.graph_from_place(place_name, network_type="all")
road_nodes, road_edges = ox.graph_to_gdfs(road_graph)

# Rename highway column before joining
road_edges = road_edges.rename(columns={"highway": "road_highway"})

# 2. Load tram tracks
tram_graph = ox.graph_from_place(
    place_name, network_type="all", custom_filter='["railway"="tram"]'
)
_, tram_edges = ox.graph_to_gdfs(tram_graph)

# Convert to the same CRS (for spatial operations)
road_edges = road_edges.to_crs(epsg=3857)
tram_edges = tram_edges.to_crs(epsg=3857)

# 3. Find intersections between roads and tram tracks
intersections = gpd.sjoin(road_edges, tram_edges, predicate="intersects")

# 4. Extract intersection points
def classify_intersection(road_type):
    if not road_type or road_type is None:
        return "Other"

    if isinstance(road_type, list):  # Handle lists properly
        road_types = [r.strip().lower() for r in road_type if isinstance(r, str)]
    else:
        road_types = [road_type.strip().lower()] if isinstance(road_type, str) else []

    # Classification rules
    traffic_light_types = {
        "motorway",
        "primary",
        "secondary",
        "tertiary",
        "primary_link",
        "secondary_link",
        "tertiary_link",
        "motorway_link",
    }
    uncontrolled_types = {"residential", "unclassified", "service"}
    pedestrian_types = {"footway", "pedestrian", "path", "cycleway"}

    if any(r in traffic_light_types for r in road_types):
        return "Traffic Light Intersection"
    elif any(r in uncontrolled_types for r in road_types):
        return "Uncontrolled Intersection"
    elif any(r in pedestrian_types for r in road_types):
        return "Pedestrian Crossing"
    return "Other"


# Iterate over the intersections and classify
intersection_points = []
intersection_types = []

for _, row in intersections.iterrows():
    geom = row.geometry
    road_type = row.get("road_highway", "Other")  # Use renamed column

    if isinstance(geom, LineString):  # Extract points from LineString geometry
        for coord in geom.coords:
            point = Point(coord)
            intersection_type = classify_intersection(road_type)

            intersection_points.append(point)
            intersection_types.append(intersection_type)

# Convert to GeoDataFrame
intersections_gdf = gpd.GeoDataFrame(
    {"type": intersection_types}, geometry=intersection_points, crs="EPSG:3857"
)

# Filter intersections within tram buffer
buffer_size = 0.2
tram_buffer = tram_edges.buffer(buffer_size)
filtered_intersections = intersections_gdf[
    intersections_gdf.geometry.within(tram_buffer.unary_union)
].to_crs(epsg=4326)

# Remove duplicates
filtered_intersections = filtered_intersections.drop_duplicates()

# Save final classified intersections
filtered_intersections.to_file(
    "data/road_tram_intersections.geojson", driver="GeoJSON"
)

# 5. Display intersections on a Folium map with colors
color_map = {
    "Traffic Light Intersection": "red",
    "Uncontrolled Intersection": "blue",
    "Pedestrian Crossing": "green",
    "Other": "gray",
}

center_lat, center_lon = (
    filtered_intersections.geometry.y.mean(),
    filtered_intersections.geometry.x.mean(),
)
m = folium.Map(
    location=[center_lat, center_lon], zoom_start=14, tiles="CartoDB Positron"
)

for _, row in filtered_intersections.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color=color_map.get(row["type"], "gray"),
        fill=True,
        fill_color=color_map.get(row["type"], "gray"),
        fill_opacity=0.7,
        tooltip=row["type"],
    ).add_to(m)

# Save map
map_file = "road_tram_intersections.html"
m.save(map_file)

# Optional: Show in Jupyter Notebook
m

/var/folders/bf/yx_hc41d1jl40qt0032h3_300000gn/T/ipykernel_48148/1474126032.py:87: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  intersections_gdf.geometry.within(tram_buffer.unary_union)


In [27]:
import json
import geopandas as gpd
import folium
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import fcluster, linkage
from shapely.geometry import Point
from collections import Counter


# Load GeoJSON file
def load_geojson(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


# Save clustered data as GeoJSON
def save_geojson(data, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


# Function to cluster points based on distance and retain intersection types
def cluster_points_by_distance(features, max_distance=0.0005):
    """
    This function will cluster points based on distance.
    max_distance is set in degrees (approximated to ~50 meters for 0.0005 degrees).
    """
    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(
        [{"type": f["properties"]["type"]} for f in features],
        geometry=[Point(f["geometry"]["coordinates"]) for f in features],
    )

    # Get the coordinates of the points
    coords = list(gdf.geometry.apply(lambda p: (p.x, p.y)))

    # Compute distance matrix (in degrees; later converted to meters)
    dist_matrix = (
        squareform(pdist(coords, metric="euclidean")) * 111139
    )  # Convert degrees to meters

    # Perform hierarchical clustering
    Z = linkage(dist_matrix, method="single")
    clusters = fcluster(Z, max_distance, criterion="distance")

    # Create a list of clustered features
    clustered_features = []
    for cluster_id in set(clusters):
        cluster_points = gdf.iloc[
            [i for i, c in enumerate(clusters) if c == cluster_id]
        ]

        # Calculate the centroid (average position) of the cluster
        avg_x = sum(p.x for p in cluster_points.geometry) / len(cluster_points.geometry)
        avg_y = sum(p.y for p in cluster_points.geometry) / len(cluster_points.geometry)

        # Get the most common intersection type in the cluster
        common_type = Counter(cluster_points["type"]).most_common(1)[0][0]

        clustered_features.append(
            {
                "type": "Feature",
                "properties": {
                    "type": common_type
                },  # The most common type within the cluster
                "geometry": {"type": "Point", "coordinates": [avg_x, avg_y]},
            }
        )

    return {"type": "FeatureCollection", "features": clustered_features}


# Example usage
input_geojson = "data/road_tram_intersections.geojson"
output_geojson = "data/clustered_intersections.geojson"

# Load and cluster data by distance
data = load_geojson(input_geojson)
clustered_data = cluster_points_by_distance(data["features"], max_distance=500)
save_geojson(clustered_data, output_geojson)
print(f"Clustered GeoJSON saved to {output_geojson}")

# Load clustered data and display on Folium map
clustered_gdf = gpd.GeoDataFrame(
    [{"type": f["properties"]["type"]} for f in clustered_data["features"]],
    geometry=[Point(f["geometry"]["coordinates"]) for f in clustered_data["features"]],
)

color_map = {
    "Traffic Light Intersection": "red",
    "Uncontrolled Intersection": "blue",
    "Pedestrian Crossing": "green",
    "Other": "gray",
}

# Compute map center
center_lat, center_lon = (
    clustered_gdf.geometry.y.mean(),
    clustered_gdf.geometry.x.mean(),
)
m = folium.Map(
    location=[center_lat, center_lon], zoom_start=14, tiles="CartoDB Positron"
)

# Plot the clusters on the map
for _, row in clustered_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color=color_map.get(row["type"], "gray"),
        fill=True,
        fill_color=color_map.get(row["type"], "gray"),
        fill_opacity=0.7,
        tooltip=row["type"],
    ).add_to(m)

# Save and display the map
map_file = "clustered_intersections.html"
m.save(map_file)
print(f"Map saved as {map_file}")
m

/var/folders/bf/yx_hc41d1jl40qt0032h3_300000gn/T/ipykernel_48148/293935697.py:44: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(dist_matrix, method="single")


Clustered GeoJSON saved to data/clustered_intersections.geojson
Map saved as clustered_intersections.html


In [30]:
import json
import pandas as pd
from collections import Counter

# Load GeoJSON file
geojson_file = "data/clustered_intersections.geojson" 
with open(geojson_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract types of intersections
types = [feature["properties"]["type"] for feature in data["features"]]

# Count occurrences
type_counts = Counter(types)

# Convert to DataFrame for better visualization
df = pd.DataFrame(type_counts.items(), columns=["Intersection Type", "Count"])

# Print table
print(df.to_string(index=False))

         Intersection Type  Count
 Uncontrolled Intersection    115
       Pedestrian Crossing    123
Traffic Light Intersection     48


In [18]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
import ast


# Function to calculate the distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters


# Load the GeoJSON file with intersections
intersections_gdf = gpd.read_file("data/clustered_intersections.geojson")

# Read the CSV event files
events_bell = pd.read_csv("data/events_BELL.csv")
events_emb1 = pd.read_csv("data/events_EMB1.csv")
events_emb2 = pd.read_csv("data/events_EMB2.csv")

# Combine all events into one DataFrame
events = pd.concat([events_bell, events_emb1, events_emb2], ignore_index=True)

# Convert the weather data to JSON format for easier extraction
events["weather"] = events["weather"].apply(ast.literal_eval)

# Define a function to calculate deceleration (m/s²)
def calculate_velocity_and_deceleration(events):
    # Convert start_time and end_time to datetime
    events["start_time"] = pd.to_datetime(events["start_time"])
    events["end_time"] = pd.to_datetime(events["end_time"])

    # Calculate duration in seconds
    events["duration_seconds"] = (
        events["end_time"] - events["start_time"]
    ).dt.total_seconds()

    # Directly rename the velocity column to initial_velocity
    events["initial_velocity"] = events[
        "velocity"
    ]  # Now `velocity` is treated as `initial_velocity`

    # Calculate final velocity (initial velocity - deceleration)
    events["final_velocity"] = (
        events["velocity"] - events["deceleration"]
    )  # Assuming final velocity is initial - deceleration

    # Calculate deceleration (m/s²), assuming deceleration is the difference in velocity over time
    events["deceleration_mps2"] = (
        (events["initial_velocity"] - events["final_velocity"]) / 3.6
    ) / events["duration_seconds"]

    events["initial_velocity_mps"] = events["initial_velocity"] / 3.6

    return events[
        [
            "vehicle",
            "lat",
            "lon",
            "initial_velocity",
            "deceleration_mps2",
            "initial_velocity_mps",
        ]
    ]


# Apply the function to the event data
events = calculate_velocity_and_deceleration(events)

# Initialize a list to store the results
results = []

# Iterate over each event to find the nearest intersection and calculate the metrics
for _, event in events.iterrows():
    nearest_intersection = None
    min_distance = float("inf")

    # Find the nearest intersection
    for _, intersection in intersections_gdf.iterrows():
        intersection_type = intersection["type"]  # Access type directly
        intersection_coords = intersection["geometry"].coords[0]
        distance = calculate_distance(
            event["lat"], event["lon"], intersection_coords[1], intersection_coords[0]
        )

        if distance < min_distance:
            min_distance = distance
            nearest_intersection = intersection_type

    # Calculate stopping distance using the formula: d = v² / (2 * a)
    if event["deceleration_mps2"] != 0:  # Prevent division by zero
        stopping_distance = (event["initial_velocity_mps"] ** 2) / (
            2 * event["deceleration_mps2"]
        )
    else:
        stopping_distance = 0  # No deceleration means no stopping distance

    # Append results for the event
    results.append(
        {
            "Intersection Type": nearest_intersection,
            "Event Count": 1,
            "Initial Speed (km/h)": event["initial_velocity"],
            "Deceleration (m/s²)": event["deceleration_mps2"],
            "Stopping Distance (m)": stopping_distance,
            "Proximity to Intersection (m)": min_distance,
        }
    )

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Group by Intersection Type and calculate the required metrics
summary_df = (
    results_df.groupby("Intersection Type")
    .agg(
        Event_Count=("Event Count", "sum"),
        Event_Count_Percentage=(
            "Event Count",
            lambda x: 100 * x.sum() / len(results_df),
        ),
        Avg_Initial_Speed=("Initial Speed (km/h)", "mean"),
        Avg_Deceleration=("Deceleration (m/s²)", "mean"),
        Avg_Stopping_Distance=("Stopping Distance (m)", "mean"),
        Avg_Proximity=("Proximity to Intersection (m)", "mean"),
    )
    .reset_index()
)

# Round the values to 2 decimal places
summary_df = summary_df.round(
    {
        "Event_Count_Percentage": 2,
        "Avg_Initial_Speed": 2,
        "Avg_Deceleration": 2,
        "Avg_Stopping_Distance": 2,
        "Avg_Proximity": 2,
    }
)

# Refining the LaTeX table to display correct column names and 2 decimal places
latex_table = summary_df.to_latex(
    index=False,
    column_format="|l|r|r|r|r|r|",
    escape=False,
    header=[
        "Intersection Type",
        "Event Count",
        "Event Count Percentage (%)",
        "Avg Initial Speed (km/h)",
        "Avg Deceleration (m/s²)",
        "Avg Stopping Distance (m)",
        "Avg Proximity to Intersection (m)",
    ],
    float_format="%.2f",
)

# Output the refined LaTeX table
print(latex_table)

\begin{tabular}{|l|r|r|r|r|r|}
\toprule
Intersection Type & Event Count & Event Count Percentage (%) & Avg Initial Speed (km/h) & Avg Deceleration (m/s²) & Avg Stopping Distance (m) & Avg Proximity to Intersection (m) \\
\midrule
Pedestrian Crossing & 33 & 36.26 & 33.30 & 1.44 & 39.17 & 34.25 \\
Traffic Light Intersection & 15 & 16.48 & 32.06 & 1.00 & 48.80 & 38.74 \\
Uncontrolled Intersection & 43 & 47.25 & 35.88 & 1.27 & 67.33 & 64.67 \\
\bottomrule
\end{tabular}



In [19]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

# Assuming the intersection and event dataframes are already loaded as intersections_gdf and event_df

# List of CSV files
event_files = ["data/events_BELL.csv", "data/events_EMB1.csv", "data/events_EMB2.csv"]
event_names = ["BELL", "EMB1", "EMB2"]

# Initialize an empty list to collect all event data with the source CSV name
event_data_list = []

for event_file, event_name in zip(event_files, event_names):
    # Load the event CSV
    event_df = pd.read_csv(event_file)

    # Add a column for the event type
    event_df["event_type"] = event_name

    # Calculate necessary metrics for each event
    for _, event in event_df.iterrows():

        # Find the nearest intersection
        nearest_intersection = None
        min_distance = float("inf")
        for _, intersection in intersections_gdf.iterrows():
            intersection_type = intersection["type"]  # Access type directly
            intersection_coords = intersection["geometry"].coords[0]
            distance = calculate_distance(
                event["lat"],
                event["lon"],
                intersection_coords[1],
                intersection_coords[0],
            )

            if distance < min_distance:
                min_distance = distance
                nearest_intersection = intersection_type

                # Get the nearest intersection using the haversine distance or other methods
                # event_point = Point(event["lon"], event["lat"])
                # nearest_intersection = intersections_gdf.geometry.distance(event_point).idxmin()
        proximity = min_distance
        intersection_type = nearest_intersection

        # Calculate stopping distance
        initial_velocity_kmh = event["velocity"]  # velocity in km/h
        initial_velocity_ms = initial_velocity_kmh / 3.6  # convert to m/s

        duration = (
            pd.to_datetime(event["end_time"]) - pd.to_datetime(event["start_time"])
        ).total_seconds()

        deceleration = initial_velocity_ms / duration
        if deceleration != 0:
            stopping_distance = (initial_velocity_ms**2) / (2 * deceleration)
        else:
            stopping_distance = (
                np.nan
            )  # If deceleration is 0, set stopping distance to NaN

        # Append results for the event
        event_data_list.append(
            {
                "Intersection Type": intersection_type,
                "Event Type": event_name,
                "Event Count": 1,
                "Avg Initial Speed (km/h)": initial_velocity_kmh,
                "Avg Deceleration (m/s²)": deceleration,
                "Avg Stopping Distance (m)": stopping_distance,
                "Avg Proximity to Intersection (m)": proximity,
            }
        )

# Create a DataFrame from the event data list
event_summary_df = pd.DataFrame(event_data_list)

# Group by intersection type and event source (BELL, EMB1, EMB2) to calculate aggregated statistics
summary_df = event_summary_df.groupby(
    ["Intersection Type", "Event Type"], as_index=False
).agg(
    {
        "Event Count": "sum",
        "Avg Initial Speed (km/h)": "mean",
        "Avg Deceleration (m/s²)": "mean",
        "Avg Stopping Distance (m)": "mean",
        "Avg Proximity to Intersection (m)": "mean",
    }
)


# Refine the LaTeX table with proper formatting
latex_table = summary_df.to_latex(
    index=False,
    column_format="|l|r|r|r|r|r|r|",
    escape=False,
    header=[
        "Intersection Type",
        "Event Type",
        "Event Count",
        "Avg Initial Speed (km/h)",
        "Avg Deceleration (m/s²)",
        "Avg Stopping Distance (m)",
        "Avg Proximity to Intersection (m)",
    ],
    float_format="%.2f",
)

# Output the refined LaTeX table
print(latex_table)

\begin{tabular}{|l|r|r|r|r|r|r|}
\toprule
Intersection Type & Event Type & Event Count & Avg Initial Speed (km/h) & Avg Deceleration (m/s²) & Avg Stopping Distance (m) & Avg Proximity to Intersection (m) \\
\midrule
Pedestrian Crossing & BELL & 13 & 37.12 & 1.08 & 60.54 & 36.83 \\
Pedestrian Crossing & EMB1 & 18 & 30.56 & 1.65 & 25.23 & 31.85 \\
Pedestrian Crossing & EMB2 & 2 & 33.05 & 1.83 & 25.81 & 39.23 \\
Traffic Light Intersection & BELL & 10 & 31.62 & 0.76 & 57.07 & 43.06 \\
Traffic Light Intersection & EMB1 & 5 & 32.94 & 1.46 & 32.26 & 30.11 \\
Uncontrolled Intersection & BELL & 26 & 36.60 & 0.71 & 88.80 & 66.16 \\
Uncontrolled Intersection & EMB1 & 13 & 34.28 & 2.08 & 33.75 & 71.32 \\
Uncontrolled Intersection & EMB2 & 4 & 36.40 & 2.22 & 36.92 & 33.36 \\
\bottomrule
\end{tabular}



In [21]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic


# Function to calculate the distance between two points (lat, lon)
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters


# Load the GeoJSON for intersections
intersections_gdf = gpd.read_file("data/clustered_intersections.geojson")

# Load the event data CSVs
bell_df = pd.read_csv("data/events_BELL.csv")
emb1_df = pd.read_csv("data/events_EMB1.csv")
emb2_df = pd.read_csv("data/events_EMB2.csv")

# Add event type column to each event DataFrame
bell_df["event_type"] = "BELL"
emb1_df["event_type"] = "EMB1"
emb2_df["event_type"] = "EMB2"

# Combine all events into one DataFrame
events_df = pd.concat([bell_df, emb1_df, emb2_df], ignore_index=True)


# Function to find the nearest intersection based on latitude and longitude
def find_nearest_intersection(event, intersections_gdf):
    min_distance = float("inf")
    nearest_intersection = None

    for _, intersection in intersections_gdf.iterrows():
        intersection_type = intersection["type"]  # Access intersection type
        intersection_coords = intersection["geometry"].coords[0]
        distance = calculate_distance(
            event["lat"], event["lon"], intersection_coords[1], intersection_coords[0]
        )

        if distance < min_distance:
            min_distance = distance
            nearest_intersection = intersection_type

    return nearest_intersection


# Add intersection types to events by finding the nearest intersection
events_df["intersection_type"] = events_df.apply(
    find_nearest_intersection, axis=1, intersections_gdf=intersections_gdf
)

# Calculate the number of occurrences of each event type per intersection type
event_counts = (
    events_df.groupby(["intersection_type", "event_type"]).size().unstack(fill_value=0)
)

# Define weights for each event type
event_weights = {
    "BELL": 1,  # Low severity
    "EMB1": 3,  # Medium severity
    "EMB2": 5,  # High severity
}

# Add the weights to the event counts and calculate the CII
event_counts["CII"] = (
    event_counts.get("BELL", 0) * event_weights["BELL"]
    + event_counts.get("EMB1", 0) * event_weights["EMB1"]
    + event_counts.get("EMB2", 0) * event_weights["EMB2"]
)

# Aggregate the CII by intersection type
intersection_type_CII = event_counts.groupby("intersection_type")["CII"].mean()

# Display the results
print("Critical Intersection Index (CII) by Intersection Type:")
print(intersection_type_CII)

# Optionally, save the event counts with CII to a CSV
event_counts.to_csv("intersection_event_counts.csv")

Critical Intersection Index (CII) by Intersection Type:
intersection_type
Pedestrian Crossing           77.0
Traffic Light Intersection    25.0
Uncontrolled Intersection     85.0
Name: CII, dtype: float64
